This product uses the FRED® API but is not endorsed or certified by the Federal Reserve Bank of St. Louis.

In [1]:
fred_series_ids = [
    "DCOILWTICO", "GASREGW", "DPROPANEMBTX", "CPIAUCSL", "PPIACO", "PCE", "CPILFESL", "UMCSENT", "PCEPILFE", "MICH",
    "PAYEMS", "UNRATE", "CES0500000003", "CIVPART", "JTSJOL", "ICSA", "ADPWNUSNERSA", "U6RATE", "AWHNONAG", "FEDFUNDS", 
    "DFEDTAR", "IOER", "DISCOUNT", "DGS10", "SOFR", "PRIME", "TB3MS", "CPFF", "CSUSHPINSA", "HSN1F", "PERMIT", "HOUST", 
    "USSTHPI", "EXHOSLUSM495S", "MORTGAGE30US", "RRVRUSQ156N", "RHORUSQ156N", "HOSINVUSM495N"
]

aaaa00001 =  ['DCOILWTICO', 'GASREGW', 'DPROPANEMBTX']
aaaa00002 =  ['CPIAUCSL', 'PPIACO', 'PCE', 'CPILFESL', 'UMCSENT', 'PCEPILFE', 'MICH']
aaaa00003 =  ['PAYEMS', 'UNRATE', 'CES0500000003', 'CIVPART', 'JTSJOL', 'ICSA', 'ADPWNUSNERSA', 'U6RATE', 'AWHNONAG']
aaaa00004 =  ['FEDFUNDS', 'DFEDTAR', 'IOER', 'DISCOUNT', 'DGS10', 'SOFR', 'PRIME', 'TB3MS', 'CPFF']
aaaa00005 =  ['CSUSHPINSA', 'HSN1F', 'PERMIT', 'HOUST', 'USSTHPI', 'EXHOSLUSM495S', 'MORTGAGE30US', 'RRVRUSQ156N', 'RHORUSQ156N', 'HOSINVUSM495N']

index_categories = {
    'energy': aaaa00001,
    'pricing': aaaa00002,
    'employment': aaaa00003,
    'federal_funds': aaaa00004,
    'housing': aaaa00005
}

number_of_years = 30

target_directory = "/Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models"


In [3]:
import fetch
import save_to_duckdb
import bronze_layer
import silver_layer_unionall
import silver_layer_binaryconversion
import silver_layer_setdatatypes
import gold_layer

dataframes, metadata_df, category_df = fetch.main(fred_series_ids, index_categories)
save_to_duckdb.main(fred_series_ids, dataframes, metadata_df, category_df)
bronze_layer.main(fred_series_ids, target_directory)
silver_layer_unionall.main(fred_series_ids, target_directory)
silver_layer_binaryconversion.main(target_directory)
silver_layer_setdatatypes.main(target_directory)
gold_layer.main(target_directory)

import documentation_reference_generation # Needs editing and needs ability to create docs folder if doesnt exist
documentation_reference_generation.main(fred_series_ids, target_directory) # Needs editing and needs ability to create docs folder if doesnt exist


SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_FactDCOILWTICO.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_FactGASREGW.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_FactDPROPANEMBTX.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_FactCPIAUCSL.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_FactPPIACO.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_FactPCE.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_FactCPILFESL.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_FactUMCSENT.sql
SQL file created: /Users/jeffreyblack/Proje

127.0.0.1 - - [17/Mar/2024 23:29:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2024 23:29:08] "GET /manifest.json?cb=1710736148737 HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2024 23:29:08] "GET /catalog.json?cb=1710736148737 HTTP/1.1" 200 -


Terminated command: dbt docs serve
